In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/skyhack-2-0/reason.csv
/kaggle/input/skyhack-2-0/customers.csv
/kaggle/input/skyhack-2-0/test.csv
/kaggle/input/skyhack-2-0/calls.csv
/kaggle/input/skyhack-2-0/sentiment_statistics.csv


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import kerastuner as kt
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense
from sklearn import metrics

In [3]:
calls = pd.read_csv('/kaggle/input/skyhack-2-0/calls.csv')
cust = pd.read_csv('/kaggle/input/skyhack-2-0/customers.csv')
reason = pd.read_csv('/kaggle/input/skyhack-2-0/reason.csv')
senti = pd.read_csv('/kaggle/input/skyhack-2-0/sentiment_statistics.csv')
test = pd.read_csv('/kaggle/input/skyhack-2-0/test.csv')

print(calls.columns)
print('\n')
print(cust.columns)
print('\n')
print(reason.columns)
print('\n')
print(senti.columns)
print('\n')
print(test.columns)


Index(['call_id', 'customer_id', 'agent_id', 'call_start_datetime',
       'agent_assigned_datetime', 'call_end_datetime', 'call_transcript'],
      dtype='object')


Index(['customer_id', 'customer_name', 'elite_level_code'], dtype='object')


Index(['call_id', 'primary_call_reason'], dtype='object')


Index(['call_id', 'agent_id', 'agent_tone', 'customer_tone',
       'average_sentiment', 'silence_percent_average'],
      dtype='object')


Index(['call_id'], dtype='object')


In [4]:
data = calls.merge(cust, on='customer_id', how='left') \
                     .merge(reason, on='call_id', how='left') \
                     .merge(senti, on='call_id', how='left')

data.head()

,call_id,customer_id,agent_id_x,call_start_datetime,agent_assigned_datetime,call_end_datetime,call_transcript,customer_name,elite_level_code,primary_call_reason,agent_id_y,agent_tone,customer_tone,average_sentiment,silence_percent_average
0,4667960400,2033123310,963118,7/31/2024 23:56,8/1/2024 0:03,8/1/2024 0:34,\n\nAgent: Thank you for calling United Airlin...,Matthew Foster,4.0,Voluntary Cancel,963118,neutral,angry,-0.04,0.39
1,1122072124,8186702651,519057,8/1/2024 0:03,8/1/2024 0:06,8/1/2024 0:18,\n\nAgent: Thank you for calling United Airlin...,Tammy Walters,NaN,Booking,519057,calm,neutral,0.02,0.35
2,6834291559,2416856629,158319,7/31/2024 23:59,8/1/2024 0:07,8/1/2024 0:26,\n\nAgent: Thank you for calling United Airlin...,Jeffery Dixon,NaN,IRROPS,158319,neutral,polite,-0.13,0.32
3,2266439882,1154544516,488324,8/1/2024 0:05,8/1/2024 0:10,8/1/2024 0:17,\n\nAgent: Thank you for calling United Airlin...,David Wilkins,2.0,Upgrade,488324,neutral,frustrated,-0.20,0.20
4,1211603231,5214456437,721730,8/1/2024 0:04,8/1/2024 0:14,8/1/2024 0:23,\n\nAgent: Thank you for calling United Airlin...,Elizabeth Daniels,0.0,Seating,721730,neutral,polite,-0.05,0.35


In [5]:
data = data.drop('agent_id_y', axis=1)
data = data.rename(columns = {'agent_id_x' : 'agent_id'})
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 14 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   call_id                  71810 non-null  int64  
 1   customer_id              71810 non-null  int64  
 2   agent_id                 71810 non-null  int64  
 3   call_start_datetime      71810 non-null  object 
 4   agent_assigned_datetime  71810 non-null  object 
 5   call_end_datetime        71810 non-null  object 
 6   call_transcript          71810 non-null  object 
 7   customer_name            71810 non-null  object 
 8   elite_level_code         46043 non-null  float64
 9   primary_call_reason      66653 non-null  object 
 10  agent_tone               71593 non-null  object 
 11  customer_tone            71810 non-null  object 
 12  average_sentiment        71701 non-null  float64
 13  silence_percent_average  71810 non-null  float64
dtypes: float64(3), int64(3

In [6]:
data.describe()

,call_id,customer_id,agent_id,elite_level_code,average_sentiment,silence_percent_average
count,7.181000e+04,7.181000e+04,71810.000000,46043.000000,71701.000000,71810.000000
mean,4.993574e+09,5.004334e+09,564768.278039,1.370871,-0.033519,0.285515
std,2.889673e+09,2.884255e+09,257532.362167,1.322297,0.143715,0.192031
min,1.316420e+05,1.197800e+04,102574.000000,0.000000,-1.380000,0.000000
25%,2.480013e+09,2.514618e+09,347606.000000,0.000000,-0.110000,0.130000
50%,4.989448e+09,4.999664e+09,591778.000000,1.000000,-0.020000,0.260000
75%,7.493629e+09,7.509126e+09,786323.000000,2.000000,0.050000,0.410000
max,9.999806e+09,9.999935e+09,993862.000000,5.000000,2.670000,0.980000


In [7]:
# Changing the data format
data['call_start_datetime'] = pd.to_datetime(data['call_start_datetime'])
data['call_end_datetime'] = pd.to_datetime(data['call_end_datetime'])
data['agent_assigned_datetime'] = pd.to_datetime(data['agent_assigned_datetime'])


# Calculate AHT (in seconds)
data['AHT'] = (data['call_end_datetime'] - data['call_start_datetime']).dt.total_seconds()

# Calculate AST (in seconds)
data['AST'] = (data['agent_assigned_datetime'] - data['call_start_datetime']).dt.total_seconds()


data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71810 entries, 0 to 71809
Data columns (total 16 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   call_id                  71810 non-null  int64         
 1   customer_id              71810 non-null  int64         
 2   agent_id                 71810 non-null  int64         
 3   call_start_datetime      71810 non-null  datetime64[ns]
 4   agent_assigned_datetime  71810 non-null  datetime64[ns]
 5   call_end_datetime        71810 non-null  datetime64[ns]
 6   call_transcript          71810 non-null  object        
 7   customer_name            71810 non-null  object        
 8   elite_level_code         46043 non-null  float64       
 9   primary_call_reason      66653 non-null  object        
 10  agent_tone               71593 non-null  object        
 11  customer_tone            71810 non-null  object        
 12  average_sentiment        71701 n